In [2]:
import math

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from jacques import kcqe

plt.style.use('ggplot')

In [6]:
# Data generation adapted from https://matplotlib.org/stable/gallery/mplot3d/scatter3d.html
# Fix random state for reproducibility
np.random.seed(9731)

def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

n = 500

# Generate a vector of 500 consecutive dates starting from today
start_date = pd.Timestamp.today().normalize()  # Normalize to remove time component
date_vector = pd.date_range(start=start_date, periods=n)

#Create random predictors x0 and x1
x0 = randrange(n, -2. * np.pi, 2. * np.pi)
x1 = randrange(n, -2. * np.pi, 2. * np.pi)
days = np.arange(1, 501)

#Create response variable y
y = 1.2 * days + 10. * np.cos(x0) + 0.5 * x1**2 + np.random.normal(loc=0., scale=5., size=n)

In [8]:
num_blocks = 10
batch_size = 1
num_epochs = 10

num_batches = math.ceil(num_blocks / batch_size)

In [10]:
p=3
n_param = int(p + 1)

init_param_vec = tf.constant(np.zeros(n_param, dtype=np.float32))
        
# declare variable representing parameters to estimate
param_vec_var = tf.Variable(initial_value=init_param_vec, name="param_vec", dtype=np.float32)

In [11]:
lls_ = np.zeros(num_epochs * num_batches, np.float32)
i = 0

trainable_variables = [param_vec_var]

In [12]:
epoch = 1
batch_ind = 1

In [17]:
df = pd.DataFrame({'date': date_vector, 'days': days, 'x0': x0, 'x1': x1, 'y': y})

In [14]:
from jacques.data_processing import assign_blocks, calc_diffs_all_train_blocks, train_test_pairings

In [18]:
block_list = assign_blocks(df, time_var = 'date', features = ['days', 'x0', 'x1'], target = 'y', num_blocks = num_blocks)

In [19]:
# Split block_list into features and targets
features_list = [block['features'] for block in block_list]
targets_list = [block['target'] for block in block_list]

In [20]:
diffs = calc_diffs_all_train_blocks(features_list, "difference")

In [21]:
train_test_matrix = train_test_pairings(num_blocks)

In [23]:
y_train = [block for block, keep in zip(features_list, train_test_matrix[batch_ind]) if keep]
y_test = targets_list[batch_ind]
test_block = features_list[batch_ind]

In [24]:
from jacques.kernels import kernel_weights2

In [25]:
param_dict = {'x_bw_raw': param_vec_var[:-1],
            'y_bw_raw': param_vec_var[-1]}

In [28]:
import tensorflow_probability as tfp

softplus_bijector = tfp.bijectors.Softplus()
corr_chol_bijector = tfp.bijectors.CorrelationCholesky()

In [30]:
theta_b = param_dict['x_bw_raw']
B_chol = tf.linalg.diag(softplus_bijector.forward(theta_b))

In [33]:
one_diffs = diffs[batch_ind]

In [35]:
one_diffs = tf.concat(diffs[batch_ind], axis=1)

In [39]:
one_diffs

<tf.Tensor: shape=(50, 350, 3), dtype=float32, numpy=
array([[[-1.0000000e+02,  8.9164009e+00,  3.1024235e-01],
        [-1.0100000e+02,  4.8176355e+00,  1.0059946e+00],
        [-1.0200000e+02,  9.8520355e+00,  5.2612772e+00],
        ...,
        [-4.4700000e+02, -9.3581438e-01,  3.5193257e+00],
        [-4.4800000e+02, -1.1504474e+00,  3.9883337e+00],
        [-4.4900000e+02,  9.6734715e+00, -4.2674308e+00]],

       [[-9.9000000e+01,  6.6218486e+00, -2.5981209e+00],
        [-1.0000000e+02,  2.5230837e+00, -1.9023688e+00],
        [-1.0100000e+02,  7.5574841e+00,  2.3529139e+00],
        ...,
        [-4.4600000e+02, -3.2303662e+00,  6.1096239e-01],
        [-4.4700000e+02, -3.4449992e+00,  1.0799704e+00],
        [-4.4800000e+02,  7.3789196e+00, -7.1757941e+00]],

       [[-9.8000000e+01,  1.6147349e+00, -2.7156711e-01],
        [-9.9000000e+01, -2.4840298e+00,  4.2418510e-01],
        [-1.0000000e+02,  2.5503705e+00,  4.6794677e+00],
        ...,
        [-4.4500000e+02, -8.23748

In [37]:
kernel_weights2(one_diffs, theta_b = param_dict['x_bw_raw'], kernel = 'gaussian_diag')

<tf.Tensor: shape=(50, 350), dtype=float32, numpy=
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)>